# Mahalanobis Distance
The Mahalanobis distance is a measure of the distance between a point P and a distribution D.  It is a multi-dimensional generalization of the idea of measuring how many standard deviations away P is from the mean of D. This distance is zero if P is at the mean of D, and grows as P moves away from the mean along each principal component axis. If each of these axes is re-scaled to have unit variance, then the Mahalanobis distance corresponds to standard Euclidean distance in the transformed space. The Mahalanobis distance is thus unitless and scale-invariant, and takes into account the correlations of the data set.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import logging
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
import sys

from scipy import linalg, stats

formatter = logging.Formatter(
    fmt='%(asctime)s.%(msecs)03d %(levelname)s [%(name)s] %(message)s',
    datefmt='%y%m%d@%H:%M:%S',
)

logger = logging.getLogger('pynhanes')
logger.setLevel(logging.DEBUG)
# f = logging.FileHandler('nhanes.log')
# f.setFormatter(formatter)
h = logging.StreamHandler(stream=sys.stdout)
h.setFormatter(formatter)

if not logger.hasHandlers():
    logger.addHandler(h)  # log to STDOUT or Jupyter
#     logger.addHandler(f)  # log to file

import pynhanes

210308@06:06:16.973 DEBUG [pynhanes] pynhanes package (re)loaded


In [3]:
dfs = pynhanes.data.load(datasets=['DEMO','BMX'], years=(2015, 2018))

210308@06:06:18.175 INFO [pynhanes.data] read 9971 rows x 47 cols from https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/DEMO_I.XPT
210308@06:06:19.236 INFO [pynhanes.data] read 9254 rows x 46 cols from https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/DEMO_J.XPT
210308@06:06:19.246 INFO [pynhanes.data] combined DEMO datasets: 19225 rows x 52 cols
210308@06:06:19.905 INFO [pynhanes.data] read 9544 rows x 26 cols from https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/BMX_I.XPT
210308@06:06:20.481 INFO [pynhanes.data] read 8704 rows x 21 cols from https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/BMX_J.XPT
210308@06:06:20.487 INFO [pynhanes.data] combined BMX datasets: 18248 rows x 29 cols


In [4]:
cols = ['RIDAGEYR','RIAGENDR','BMXBMI']
df = pd.concat([d.set_index('SEQN') for d in dfs.values()], axis=1, join='outer')

# gab: gender, age, bmi
gab = df.loc[df.RIDAGEYR.between(18,65), cols].dropna()
gab.shape

(8651, 3)

In [5]:
gab.head()

,RIDAGEYR,RIAGENDR,BMXBMI
SEQN,,,
83732.0,62.0,1.0,27.8
83733.0,53.0,1.0,30.8
83735.0,56.0,2.0,42.4
83736.0,42.0,2.0,20.3
83741.0,22.0,1.0,28.0


In [6]:
def mahalanobis(x=None, data=None, cov=None):
    """Compute the Mahalanobis Distance between each row of x and the data  
    x    : vector or matrix of data with, say, p columns.
    data : ndarray of the distribution from which Mahalanobis distance of each observation of x is to be computed.
    cov  : covariance matrix (p x p) of the distribution. If None, will be computed from data.
    """
    x_minus_mu = x - np.mean(data)
    if not cov:
        cov = np.cov(data.values.T)
    inv_covmat = linalg.inv(cov)
    left_term = np.dot(x_minus_mu, inv_covmat)
    mahal = np.dot(left_term, x_minus_mu.T)
    return mahal.diagonal()

gab['mahalanodis'] = mahalanobis(x=gab, data=gab)

In [7]:
gab['md_decile'] = pd.qcut(gab.mahalanodis, q=[x/10 for x in range(0, 11)], labels=range(1, 11))
gab.groupby(['md_decile']).mean()

,RIDAGEYR,RIAGENDR,BMXBMI,mahalanodis
md_decile,,,,
1,41.804850,1.629330,29.526212,1.124808
2,41.853179,1.468208,29.056185,1.421375
3,41.953757,1.472832,28.812601,1.718357
4,42.421965,1.494798,28.462081,2.033994
5,41.579191,1.539884,28.594566,2.399284
6,42.720231,1.500578,28.243006,2.785824
7,44.670520,1.530636,28.681156,3.187419
8,42.691329,1.502890,28.211792,3.637503
9,37.988439,1.495954,28.427861,4.228439


## Outlier Detection
Assuming that the Mahalanobis distance statistic follows chi-square distribution, the critical value at a 0.01 significance level and 2 degrees of freedom, computed as `stats.chi2.ppf(0.99, df=2)`, is 9.21, meaning that an observation can be considered extreme if the M distance > 9.21.  Using this threshold, we filter out 145 survey participants with extreme BMI levels (BMI > 45).

In [9]:
gab['chi2p'] = 1 - stats.chi2.cdf(gab['mahalanodis'], 2)
gab[gab.chi2p < .01]

,RIDAGEYR,RIAGENDR,BMXBMI,mahalanodis,md_decile,chi2p
SEQN,,,,,,
83963.0,44.0,2.0,52.1,9.489367,10,8.697814e-03
84022.0,60.0,2.0,58.7,16.068693,10,3.241363e-04
84230.0,28.0,2.0,57.8,16.420222,10,2.718905e-04
84277.0,28.0,2.0,51.5,10.687899,10,4.776967e-03
84396.0,38.0,2.0,59.4,16.589922,10,2.497723e-04
...,...,...,...,...,...,...
102238.0,49.0,2.0,56.5,13.229306,10,1.340580e-03
102355.0,33.0,1.0,53.7,12.800717,10,1.660962e-03
102450.0,39.0,1.0,56.1,14.359064,10,7.620245e-04


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=98f37550-cbaa-491c-a3ea-f393696dc041' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>